In [7]:
!pip3 install -r requirements.txt

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 7.8 MB/s  0:00:01 eta 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 10.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.4 MB/s  0:00:00
Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 8.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [matplotlib]1 [matplotlib]


In [19]:
import os
from datasets import load_dataset
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string
from collections import Counter
import matplotlib.pyplot as plt
import random

# Dataset load (Hugging Face)

In [5]:
# Define the local directory where you want to save the data
local_path = os.path.join(os.getcwd(), "local_wikipedia_data")

# 1. Download and save the data to the specified local_path
ds = load_dataset(
    "wikimedia/wikipedia",
    "20231101.en",
    cache_dir=local_path  
)

print(f"Dataset successfully downloaded/loaded to: {local_path}")


Generating train split: 100%|██████████| 6407814/6407814 [04:31<00:00, 23604.73 examples/s]


Dataset successfully downloaded/loaded to: /home/san22chit/Documents/IIITH/Sem3/IRE/Assignments/IndexingAndRetrieval/local_wikipedia_data


# Part 1

## Preprocessing (Hugging Face Dataset)

In [16]:
# Download NLTK resources if not already present
nltk.download('punkt_tab')
nltk.download('stopwords')


# Get English stopwords and initialize stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
punct_table = str.maketrans('', '', string.punctuation)

def preprocess(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(punct_table)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords and stem
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words and word.isalpha()]
    return tokens


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/san22chit/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/san22chit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def get_word_counts(split, preprocess_func=None, max_docs=10000):
    word_count = Counter()
    for idx, item in enumerate(ds[split]):
        text = item['text']
        if preprocess_func:
            tokens = preprocess_func(text)
        else:
            tokens = word_tokenize(text)
        word_count.update(tokens)
        if idx+1 >= max_docs:
            break
    return word_count

def plot_word_freq(counter, title, filename, top_n=30):
    most_common = counter.most_common(top_n)
    words, counts = zip(*most_common)
    plt.figure(figsize=(14,7))
    plt.bar(words, counts)
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

In [ ]:
split = 'train'
max_docs = 50000  # adjust as needed

# Original (no preprocessing)
print("Counting words without preprocessing...")
original_counts = get_word_counts(split, preprocess_func=None, max_docs=max_docs)
plot_word_freq(original_counts, "Word Frequency Before Preprocessing", "freq_before.png")

# Preprocessed
print("Counting words with preprocessing...")
processed_counts = get_word_counts(split, preprocess_func=preprocess, max_docs=max_docs)
plot_word_freq(processed_counts, "Word Frequency After Preprocessing", "freq_after.png")

print("Plots saved as freq_before.png and freq_after.png")

Counting words without preprocessing...
Counting words with preprocessing...
Plots saved as freq_before.png and freq_after.png
